## Exploring Battle Logs

In [1]:
from bs4 import BeautifulSoup



In [3]:
INPUT_DIR="../battle_data"
OUTPUT_DIR="../battle_data_parsed/v1"

with open(INPUT_DIR + "/0001.html") as fp:
    soup = BeautifulSoup(fp, "html.parser")


In [40]:
UNKNOWN_POKEMON_ID_PREFIX = 'UNKNOWN_POKEMON'

class Pokemon:
    def __init__(self, player, id, logs):
        self._id = id
        self._player = player
        self._logs = logs


In [41]:
class PokemonTeam:
    def __init__(self, player, logs):
        self._player = player
        self._logs = logs
        self._pokemon = {}


    def __len__(self):
        for log in self._logs:
            if log[0] == 'teamsize' and log[1] == self._player:
                return int(log[2])

        return 0


    def _load_pokemon(self):
        count = len(self)
        found_count = 0

        for log in self._logs:
            if log[0] == 'switch' and log[1].startswith(self._player + 'a:'):
                p_id = log[2]
                if p_id in self._pokemon:
                    continue

                pokemon = Pokemon(player=self._player, id=p_id, logs=self._logs)
                self._pokemon[pokemon] = pokemon
                found_count += 1

        while found_count < count:
            p_id = UNKNOWN_POKEMON_ID_PREFIX + "_" + str(count - found_count)
            pokemon = Pokemon(player=self._player, id=p_id, logs=self._logs)
            self._pokemon[p_id] = pokemon
            found_count += 1




In [42]:
class BattleLogs:
    def __init__(self, logs):
        self._logs = logs

    
    @property
    def tier(self):
        for log in self._logs:
            if log[0] == "tier":
                return log[1]

    @property
    def is_rated(self):
        for log in log:
            if log[0] == 'rated':
                return True

        return False

    
    @property
    def rules(self):
        return [x[1] for x in self._logs if x[0] == 'rule']


    @property
    def p1_team(self):
        return PokemonTeam(player='p1', logs=self._logs)


    @property
    def p2_team(self):
        return PokemonTeam(player='p2', logs=self._logs)



In [49]:
el = soup.find_all('script', class_='battle-log-data')[0]

raw_logs = [x.split('|')[1:] for x in el.text.split('\n') if len(x) > 0]

logs = BattleLogs(raw_logs)


In [50]:
team1 = logs.p1_team
team1._load_pokemon()
team1._pokemon

{<__main__.Pokemon at 0x7fd7b89c7790>: <__main__.Pokemon at 0x7fd7b89c7790>,
 <__main__.Pokemon at 0x7fd7b8c10d30>: <__main__.Pokemon at 0x7fd7b8c10d30>,
 <__main__.Pokemon at 0x7fd7b8c102e0>: <__main__.Pokemon at 0x7fd7b8c102e0>,
 <__main__.Pokemon at 0x7fd7b8c105b0>: <__main__.Pokemon at 0x7fd7b8c105b0>,
 <__main__.Pokemon at 0x7fd7b8c106d0>: <__main__.Pokemon at 0x7fd7b8c106d0>}

In [55]:
for log in raw_logs:
    if log[0] == 'switch' and log[1].startswith('p2a'):
        print(log)

['switch', 'p2a: Duraludon', 'Duraludon, L84, F', '255\\/255']
['switch', 'p2a: Archeops', 'Archeops, L82, M', '257\\/257']
['switch', 'p2a: Gothitelle', 'Gothitelle, L86, F', '261\\/261']
['switch', 'p2a: Regidrago', 'Regidrago, L78', '440\\/440']
['switch', 'p2a: Darmanitan', 'Darmanitan-Galar, L78, M', '292\\/292']
['switch', 'p2a: Lunala', 'Lunala, L72', '316\\/316']
